## Configuration

In [ ]:
### RUN THIS CELL TO BEGIN CONFIGURATION ###

%run configuration_form.ipynb

In [ ]:
### LEGACY CONFIGURATION CELL -- SKIP THIS IF USING THE CONFIGURATION FORM ABOVE ###

## CHOOSE YOUR ACTION. Options are "import", "delete", and "update".
action = 'import'


## CONFIGURE IMPORT FILES

"""
List paths to CSV or Excel files to insert. Files must contain the following headers in order:

_id, publication, description, publisher, edition,contentType,language,country,
startdate,enddate,altTitle,authors

Excel spreadsheets must contain data only in sheet 1.
"""
source_files = ['path_to_source_files']


## CONFIGURE `_id` TO DELETE A MANIFEST
delete_id = '' # A string


## CONFIGURE `_id` AND PROPERTIES TO UPDATE A MANIFEST
"""
Properties must be given in Python dict form. E.g.:

{'description': 'Some other text', 'edition': 'online'}

"""
# Configure Database ID and Properties to Update
update_id = '' # A string
properties = {'key1': 'value1', 'key2': 'value2'} # A dict

## Basic Setup

In [ ]:
# If using the configuration form, get the values from the form
try:
    action = config.values['action']
    delete_id = config.values['delete_id']
    update_id = config.values['update_id']
    properties = config.values['properties']
except:
    pass

# Import dependencies
import os, datetime, tabulator, itertools
import pymongo
from pymongo import MongoClient
import pandas as pd
from tableschema_pandas import Storage

# Set up the MongoDB client, configure the databases, and assign variables to the "collections" 
client = MongoClient('mongodb://localhost:27017')
db = client.we1s
publications = db.Publications
we1s_log = db.we1s_log

# Set up the storage functions for pandas dataframes
storage = Storage()
storage.create('data', {
    'primaryKey': '_id',
    'fields': [
        {'name': '_id', 'type': 'string'},
        {'name': 'publication', 'type': 'string'},
        {'name': 'description', 'type': 'string'},
        {'name': 'publisher', 'type': 'string'},
        {'name': 'edition', 'type': 'string'},
        {'name': 'contentType', 'type': 'string'},
        {'name': 'language', 'type': 'string'},
        {'name': 'country', 'type': 'string'},
        {'name': 'startdate', 'type': 'string'},
        {'name': 'enddate', 'type': 'string'},
        {'name': 'altTitle', 'type': 'string'},
        {'name': 'authors', 'type': 'string'}
    ]
})


## API Methods

In [ ]:
def clear(collection):
    """
    Removes all documents from the specified collection.
    """
    collection.delete_many({})
    print('All documents in the specified collection have been deleted.')


def delete_publication(id):
    """
    Deletes a publication manifest based on id.
    """
    result = publications.delete_one({'_id': id})
    if result.deleted_count != 0:
        we1s_log.insert_one({'id': id, 'date': datetime.datetime.utcnow(), 'type': 'delete'})
        print('Document "' + str(id) + ' was deleted.')
    else:
        print('Error: The document could not be deleted. Make sure you have the correct "id" by running `list_publications()`.')
    

def get_page(pages, page):
    """
    Takes a list of paginated results form `paginate()` and returns a single page from the list.
    """
    try:
        return pages[page-1]
    except:
        print('The requested page does not exist.')
    
    
def import_manifests(source_files):
    """
    Loops through the source files and streams them into a dataframe, then converts
    the dataframe to a list of manifest dicts.
    """    
    for item in source_files:
        if item.endswith('.xlsx') or item.endswith('.xls'):
            options = {'format': 'xlsx', 'sheet': 1, 'headers': 1}
        else:
            options = {'headers': 1}
        try:
            with tabulator.Stream(item, **options) as stream:
                try:
                    validate_headers(stream.headers)
                    storage.write('data', stream)
                except:
                    print('Error: The table headings in ' + item + ' do not match the Publications schema.')
        except:
            print('Error: Could not stream tabular data.')
    manifests = []
    for key, properties in storage['data'].to_dict('index').items():
        properties['_id'] = key
        properties['date'] = []
        try:
            assert len(properties['enddate']) > 0
            start = properties.pop('startdate', None)
            end = properties.pop('enddate', None)
            properties['date'].append({"start": start})
            properties['date'].append({"end": end})
        except:
            properties['date'].append(properties['startdate'])
    manifests.append(properties)
    return manifests

    
def insert_publication(manifest):
    """
    Inserts a publication manifest after checking for a unique `_id`.
    """
    try:
        assert manifest['_id'] not in publications.distinct("_id")
        publications.insert_one(manifest)
        we1s_log.insert_one({'id': manifest['_id'], 'date': datetime.datetime.utcnow(), 'type': 'insert'})
        print('Inserted manifest with `_id` "' + manifest['_id'] + '".')
    except:
        print('The `_id` "' + manifest['_id'] + '" already exists in the database.')

    
def list_publications(page_size=10, page=1):
    """
    Prints a list of all publications.
    """
    if len(list(publications.find())) > 0:
        result = list(publications.find())
        pages = list(paginate(result, page_size=page_size))
        page = get_page(pages, page)
        print(page)
    else:
        print('The Publications database is empty.')
    

def paginate(iterable, page_size):
    """
    Returns a generator with a list sliced into pages by the designated size. If 
    the generator is converted to a list called `pages`, and individual page can 
    be called with `pages[0]`, `pages[1]`, etc.
    """
    while True:
        i1, i2 = itertools.tee(iterable)
        iterable, page = (itertools.islice(i1, page_size, None),
                list(itertools.islice(i2, page_size)))
        if len(page) == 0:
            break
        yield page

        
def search(values):
    """
    Returns search results.
    """
    print(values)
    if len(list(publications.find())) > 0:
        if value['regex'] == True:
            query = {}
            for k, v in value['query'].items():
                REGEX = re.compile(v)
                query[k] = {'$regex': REGEX}
        else:
            query = values['query']
            result = list(publications.find(
                query,
                limit=values['limit'],
                projection=values['show_properties']))
            pages = list(paginate(result, page_size=page_size))
            page = get_page(pages, page)
            print(page)
    else:
        print('The Publications database is empty.')
        
        
def show_databases():
    """
    Lists all databases in the current client.
    """
    if len(client.database_names()) > 0:
        print(client.database_names())
    else:
        print('The WE1S database is empty.')

    
def show_log():
    """
    Prints the log of database transactions.
    """
    print(list(we1s_log.find()))

    
def update_publication(id, properties):
    """
    Updates a publication manifest based on id. Takes a dict containing all the properties to be updated.
    """
    publications.update_one({"_id": id}, {"$set": properties}, upsert=False)
    we1s_log.insert_one({'id': id, 'date': datetime.datetime.utcnow(), 'type': 'update'})
    print('The manifest for `_id` "' + id + '" has been updated.')
    

def validate_headers(headers):
    """
    Verifies that the headers in the tabular stream match the Publications schema.
    """
    assert headers == ['_id', 'publication', 'description', 'publisher', 'edition', 
          'contentType', 'language', 'country', 'startdate', 
          'enddate', 'altTitle', 'authors']

## Execute Action

In [ ]:
# Call relevant functions based on action configuration
if action == 'import':
    print('Processing...')
    for manifest in import_manifests(source_files):
        insert_publication(manifest)
elif action == 'Delete':    
    delete_publication(delete_id)
elif action == 'Update':
    update_publication(update_id, properties)
elif action == 'Search':
    search(config.values)
else:
    print('Please set the `action` configuration at the top of the notebook.')

## Extra Function Calls

In [ ]:
# Run these API methods if desired

# list_publications()
# show_log()
# clear(publications)
# clear(we1s_log)